In [1]:
# libraries
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
# our modules
from preprocessing import *

In [2]:
torneo = pd.read_csv('data/prova.csv', 
                            encoding='utf-8-sig', 
                            dtype=DATA_TYPES,
                            parse_dates=['Date', 'WBD', 'LBD'])
torneo

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,WElo,WSurfElo,WBD,WHand,WBHand,LElo,LSurfElo,LBD,LHand,LBHand
0,2,Doha,Qatar Exxon Mobil Open,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Herbert P.H.,...,NaN,NaN,1991-03-18,R,2.0,2066.2,2003.9,1993-09-03,R,1.0
1,2,Doha,Qatar Exxon Mobil Open,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Fucsovics M.,...,1747.4,1712.9,1992-02-08,R,2.0,1668.8,1639.6,1990-10-17,R,1.0
2,2,Doha,Qatar Exxon Mobil Open,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Djokovic N.,...,2201.3,2142.4,1987-05-22,R,2.0,1723.9,1682.0,1992-05-20,R,2.0
3,2,Doha,Qatar Exxon Mobil Open,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Cecchinato M.,...,1626.7,1496.9,1992-09-30,R,1.0,1537.7,1481.7,1986-01-06,R,1.0
4,2,Doha,Qatar Exxon Mobil Open,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Lajovic D.,...,1716.2,1649.5,1990-06-30,R,1.0,1782.0,1727.6,1988-06-29,L,2.0
5,2,Doha,Qatar Exxon Mobil Open,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Garcia-Lopez G.,...,NaN,NaN,1983-06-04,R,1.0,NaN,NaN,1995-04-14,R,2.0
6,2,Doha,Qatar Exxon Mobil Open,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Wawrinka S.,...,1920.7,1884.8,1985-03-28,R,1.0,1872.7,1817.9,1996-05-21,R,2.0


In [7]:
def simulation(atp = 1, 
               min_date=2011,
               max_date=2019,
               features_to_drop=[], 
               missing_values="drop", 
               drop_first=False):
    
    tournament = pd.read_csv("data/" + str(max_date+1) + ".csv", encoding='utf-8-sig', dtype=DATA_TYPES)
    tournament = tournament[tournament['ATP'] == atp]
    # TODO filtrare per atp e round(primo)
    
    tournament_to_test = unify_data(tournament, features_to_drop, 'drop', drop_first)
    
    X, Y = preprocess_data(min_date, max_date, features_to_drop, missing_values, drop_first)
    
    # TODO allenare e simulare su tournament
    
    dt = DecisionTreeClassifier(max_leaf_nodes=200,
                                splitter='best',
                                criterion='entropy',
                                max_depth=15)
    dt.fit(X, Y)
    
    prediction = dt.predict(tournament_to_test[tournament_to_test['Round'] == 0])
    
    firstR = tournament[tournament['Round'] == '1st Round']
    i = 0
    for j, match in firstR.iterrows():
        print('P1: ' + match['Winner'], 'P2: '+ match['Loser'], 'Wins:', prediction[i])
        i+=1

    print(prediction)
    
    return tournament, X, Y
    

In [9]:
T, X, Y = simulation(atp = 2, min_date=2014, max_date=2019, features_to_drop=['Best of', 'Court', 'Surface'])

Elo rankings computing...


KeyError: "['EXW' 'EXL' 'LBW' 'LBL' 'SJW' 'SJL' 'WBD' 'LBD'] not found in axis"

In [ ]:
T